# Open Earth Foundation
## Open Ocean

This is the second notebook of a series of 5 notebooks that explains step by step how to calculate each modulating factor and assign credtis for the [Marine Biodiversity Credits methodology](https://zenodo.org/records/10182712) applied to the Cocos Marine Conservation Area of Costa Rica

## Libraries

In [ ]:
#!pip install pyobis

In [ ]:
import os
import glob
import boto3

import pyproj
import math
import numpy as np
import pandas as pd
from pyobis import occurrences

import matplotlib.pyplot as plt

import shapely
import geopandas as gpd
from shapely.geometry import Polygon, Point, box
from shapely.ops import linemerge, unary_union, polygonize

In [ ]:
import fiona; #help(fiona.open)

In [ ]:
%load_ext autoreload

In [ ]:
#Run this to reload the python file
%autoreload 2
from MBU_utils import *

# Data

Two data sources are used for these calculations, data from OBIS (observation points) and those from the IUCN red list (distribution polygons).

- Indices such as Shannon and Simpson can be calculated only with OBIS data
- Metrics like WEGE and Endemism can be calculated only with IUCN data
- Habitats accounting is calculating with IUCN Global Ecosystem Typology

**Accessing OBIS data**

OBIS data contains species information and points of observation

In [ ]:
# MPA file
ACMC = gpd.read_file('https://ocean-program.s3.amazonaws.com/data/raw/MPAs/ACMC.geojson')

In [ ]:
#create a polygon to access the OBIS data
min_x, min_y, max_x, max_y = ACMC.total_bounds
geometry = f"POLYGON(({max_x} {min_y}, {min_x} {min_y}, {min_x} {max_y}, {max_x} {max_y}, {max_x} {min_y}))"

query = occurrences.search(geometry=geometry)
query.execute()

# Returns the data
df2 = query.data 

In [ ]:
# convert OBIS dataframe to geodataframe
# making point with the lat and lon information
df2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.decimalLongitude, df2.decimalLatitude))

In [ ]:
df2.crs = "EPSG:4326"

In [ ]:
df2.crs

**Accesing IUCN RedList data**

In [ ]:
df1 = gpd.read_file('s3://ocean-program/data/processed/ACMC_IUCN_RedList/gdf_ACMC_IUCN_range_status_filtered.shp')

In [ ]:
df1.crs = "EPSG:4326"

In [ ]:
df1.crs

**Accesing IUCN Global Ecosystem Typology**

For this step, it's needed only the ubication path of these files

In [ ]:
#From AWS
s3 = boto3.resource('s3')
bucket = s3.Bucket('ocean-program')

eco_names = []

for obj in bucket.objects.filter(Prefix='data/raw/Ecosystems/'):
    name = obj.key[:]
    eco_names.append(f'https://ocean-program.s3.amazonaws.com/{name}')

# Metrics and Indices

First, we need to build a grid geoDataFrame

In [ ]:
# Making a grid geoDataFrame
grid = create_grid(ACMC, grid_shape="hexagon", grid_size_deg=0.05)

In [ ]:
grid.crs

## Using OBIS data

`Note`: Endemism and WEGE indices are not available for this data set

**Shannon Index**

In [ ]:
Shannon = shannon(ACMC, df2, grid)

In [ ]:
Shannon.head()

In [ ]:
Shannon = gpd.clip(Shannon.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))
ax = Shannon.plot(column='shannon', figsize=(6, 6), cmap='viridis', edgecolor="grey", legend = True)

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none', label='ACMC')
plt.title('Shannon Index for ACMC\nSource: OBIS')

**Simpson Index**

In [ ]:
Simpson = simpson(ACMC, df2, grid)

In [ ]:
Simpson.head()

In [ ]:
Simpson = gpd.clip(Simpson.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))
ax = Simpson.plot(column='simpson', figsize=(6, 6), cmap='viridis', edgecolor="grey", legend = True)

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none', label='ACMC')
plt.title('Simpson Index for ACMC\nSource: OBIS')

**Species Richness**

In [ ]:
Species_Richness = species_richness(ACMC, df2, grid)

In [ ]:
Species_Richness.head()

In [ ]:
Species_Richness = gpd.clip(Species_Richness.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))
ax = Species_Richness.plot(column='species_richness', figsize=(6, 6), cmap='viridis', edgecolor="grey", legend = True)

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none', label='ACMC')
plt.title('Species Richness metric for ACMC\nSource: OBIS')

## Using IUCN data

`Note`: Shannon and Simpson indices are not available for this data set

**Species Richness**

In [ ]:
Species_Richness = species_richness(ACMC, df1, grid)

In [ ]:
Species_Richness.head()

In [ ]:
Species_Richness = gpd.clip(Species_Richness.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))
ax = Species_Richness.plot(column='species_richness', figsize=(6, 6), cmap='viridis', edgecolor="grey", legend = True)

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none', label='ACMC')
plt.title('Species Richness metric for ACMC\nSource: IUCN')

**Endemism**

In [ ]:
Endemism = endemism(ACMC, df1, grid)

In [ ]:
Endemism.head()

In [ ]:
Endemism = gpd.clip(Endemism.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))
ax = Endemism.plot(column='endemism', figsize=(6, 6), cmap='viridis', edgecolor="grey", legend = True)

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none', label='ACMC')
plt.title('Endemism metric for ACMC\nSource: IUCN')

**WEGE Index**

In [ ]:
Wege = wege(ACMC, df1, grid)

In [ ]:
Wege.head()

In [ ]:
Wege = gpd.clip(Wege.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))
ax = Wege.plot(column='wege', figsize=(6, 6), cmap='viridis', edgecolor="grey", legend = True)

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none', label='ACMC')
plt.title('Wege index for ACMC\nSource: IUCN')

**Habitats Accounting**

In [ ]:
Habitat_accounting = habitat_accounting(ACMC, grid, eco_names)

In [ ]:
Habitat_accounting.head()

In [ ]:
Habitat_accounting = gpd.clip(Habitat_accounting.set_crs(epsg=4326, allow_override=True), ACMC.set_crs(epsg=4326, allow_override=True))
ax = Habitat_accounting.plot(column='habitat_accounting', figsize=(6, 6), cmap='viridis', edgecolor="grey", legend = True)

gpd.GeoSeries(ACMC.geometry).plot(ax=ax, edgecolor='black', facecolor='none', label='ACMC')
plt.title('Habitat_accounting for ACMC\nSource: IUCN')